In [111]:
import polars as pl
import polars.selectors as cs
import os

In [112]:
# Initial Tasks.  Perform each of the following tasks before constructing the two data sets.

# 1.	Create a lab 5.3 Python jupyter notebook in the root directory of the module 5 lab repository,
# 2.	Use glob to verify that you have the pitching, fielding, and salaries tables downloads, and
# 3.	Explore the Salaries.csv file to determine the latest year with salary data.

# Construct the Salary dataset.  Our goal is to build a single data set covering 11 years including 
# the most recent year with salary data as well as a decade of previous data.  
# The table will have one row per player-year and needs to contain the following

# 1.	The cases will be all pitchers from the years determined above.
# 2.	The target column is the pitcher’s yearly salary, which can be found in the Salaries.csv file.
# 3.	The features for this problem will be a combination the pitching and fielding statistics found in the respective files.
#   Be sure to perform a grouped aggregation to combine data for all players that were traded
#  (e.g., had multiple stints in a year).
# 4.	We will also want to create a Training-Validation column, which will be marked as Validation for the most recent
#  complete season and Training otherwise.



In [113]:
import glob

csv_files = glob.glob("data/lahman_1871-2024_csv/*.csv")

csv_files


['data/lahman_1871-2024_csv\\AllstarFull.csv',
 'data/lahman_1871-2024_csv\\Appearances.csv',
 'data/lahman_1871-2024_csv\\AwardsManagers.csv',
 'data/lahman_1871-2024_csv\\AwardsPlayers.csv',
 'data/lahman_1871-2024_csv\\AwardsShareManagers.csv',
 'data/lahman_1871-2024_csv\\AwardsSharePlayers.csv',
 'data/lahman_1871-2024_csv\\Batting.csv',
 'data/lahman_1871-2024_csv\\BattingPost.csv',
 'data/lahman_1871-2024_csv\\CollegePlaying.csv',
 'data/lahman_1871-2024_csv\\Fielding.csv',
 'data/lahman_1871-2024_csv\\FieldingOF.csv',
 'data/lahman_1871-2024_csv\\FieldingOFsplit.csv',
 'data/lahman_1871-2024_csv\\FieldingPost.csv',
 'data/lahman_1871-2024_csv\\HallOfFame.csv',
 'data/lahman_1871-2024_csv\\HomeGames.csv',
 'data/lahman_1871-2024_csv\\Managers.csv',
 'data/lahman_1871-2024_csv\\ManagersHalf.csv',
 'data/lahman_1871-2024_csv\\Parks.csv',
 'data/lahman_1871-2024_csv\\People.csv',
 'data/lahman_1871-2024_csv\\Pitching.csv',
 'data/lahman_1871-2024_csv\\PitchingPost.csv',
 'data/lahm

In [114]:

(salaries :=
 pl.read_csv('./data/lahman_1871-2024_csv/Salaries.csv')
      .filter(pl.col('yearID').is_between(2006, 2016))
)

salaries


yearID,teamID,lgID,playerID,salary
i64,str,str,str,i64
2007,"""CHA""","""AL""","""aardsda01""",387500
2008,"""BOS""","""AL""","""aardsda01""",403250
2009,"""SEA""","""AL""","""aardsda01""",419000
2010,"""SEA""","""AL""","""aardsda01""",2750000
2011,"""SEA""","""AL""","""aardsda01""",4500000
…,…,…,…,…
2010,"""DET""","""AL""","""zumayjo01""",915000
2011,"""DET""","""AL""","""zumayjo01""",1400000
2014,"""SEA""","""AL""","""zuninmi01""",504100


#### latest year for the salary data is 2016

In [115]:
pitching = (
    pl.read_csv('./data/lahman_1871-2024_csv/Pitching.csv')
      .filter(pl.col('yearID').is_between(2006, 2016))
      .group_by(["playerID", "yearID"])
      .agg([
          pl.col("W").sum(),
          pl.col("L").sum(),
          pl.col("G").sum(),
          pl.col("GS").sum(),
          pl.col("SV").sum(),
          pl.col("IPouts").sum(),
          pl.col("H").sum(),
          pl.col("ER").sum(),
          pl.col("HR").sum(),
          pl.col("BB").sum(),
          pl.col("SO").sum()
      ])
)
pitching

playerID,yearID,W,L,G,GS,SV,IPouts,H,ER,HR,BB,SO
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""rupejo01""",2008,3,1,46,0,0,268,93,51,8,46,53
"""chulkvi01""",2012,1,0,7,0,0,27,17,10,0,4,10
"""ellinbr02""",2016,4,2,32,0,0,99,27,9,2,16,32
"""roberke02""",2015,1,1,15,0,0,41,22,11,0,3,6
"""delatjo01""",2013,0,0,7,0,0,34,10,8,2,10,15
…,…,…,…,…,…,…,…,…,…,…,…,…
"""raburry01""",2013,0,0,1,0,0,3,0,0,0,0,1
"""smokejo01""",2016,3,0,20,0,0,46,16,8,4,4,25
"""marcush01""",2010,13,8,31,31,0,586,181,79,24,43,165


In [116]:
(salaries :=
    pl.read_csv('./data/lahman_1871-2024_csv/Salaries.csv')
      .filter(pl.col('yearID').is_between(2006, 2016))
      .select(['playerID', 'yearID', 'salary'])
)

salaries

playerID,yearID,salary
str,i64,i64
"""aardsda01""",2007,387500
"""aardsda01""",2008,403250
"""aardsda01""",2009,419000
"""aardsda01""",2010,2750000
"""aardsda01""",2011,4500000
…,…,…
"""zumayjo01""",2010,915000
"""zumayjo01""",2011,1400000
"""zuninmi01""",2014,504100


In [117]:
(fielding_p :=
    pl.read_csv('./data/lahman_1871-2024_csv/Fielding.csv')
      .filter(pl.col('yearID').is_between(2006, 2016))
      .filter(pl.col('POS') == 'P')
      .group_by(['playerID', 'yearID'])
      .agg([
          pl.col('PO').sum(),
          pl.col('A').sum(),
          pl.col('E').sum(),
          pl.col('G').sum().alias('G_field')
      ])
)

fielding_p



playerID,yearID,PO,A,E,G_field
str,i64,i64,i64,i64,i64
"""achteaj01""",2015,1,0,1,11
"""horstje01""",2013,0,4,0,28
"""hollade01""",2014,1,2,1,6
"""dillati01""",2009,1,0,0,2
"""meyerda02""",2009,2,2,2,71
…,…,…,…,…,…
"""millean01""",2009,6,8,1,20
"""belltr01""",2009,1,1,0,8
"""oliveda02""",2006,4,10,0,45


In [118]:
(features :=
    pitching
      .join(fielding_p, on=['playerID', 'yearID'], how='left')
)

features


playerID,yearID,W,L,G,GS,SV,IPouts,H,ER,HR,BB,SO,PO,A,E,G_field
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""rupejo01""",2008,3,1,46,0,0,268,93,51,8,46,53,5,10,0,46
"""chulkvi01""",2012,1,0,7,0,0,27,17,10,0,4,10,1,0,0,7
"""ellinbr02""",2016,4,2,32,0,0,99,27,9,2,16,32,0,2,0,32
"""roberke02""",2015,1,1,15,0,0,41,22,11,0,3,6,1,2,0,15
"""delatjo01""",2013,0,0,7,0,0,34,10,8,2,10,15,1,0,0,7
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""raburry01""",2013,0,0,1,0,0,3,0,0,0,0,1,1,0,0,1
"""smokejo01""",2016,3,0,20,0,0,46,16,8,4,4,25,0,1,0,20
"""marcush01""",2010,13,8,31,31,0,586,181,79,24,43,165,18,24,0,31


In [ ]:


(final_ds :=
    features
      .join(salaries, on=["playerID", "yearID"], how="left") 
      .with_columns(
          pl.when(pl.col("yearID") == 2016)
            .then(pl.lit("Validation"))
            .otherwise(pl.lit("Training"))
            .alias("TrainingValidation")
      )
      .sort(["yearID", "playerID"])
)

final_ds


In [ ]:
final_ds.group_by("TrainingValidation").agg(pl.len().alias("n"))
